# Single neuron

## 1. Regular Spiking

In [ ]:
from brian2 import *


### Cell parameters ###
S       = 20000.* um2 # Area of membrane
gl      = 0.05 * msiemens/ cm2# Membrane time constant
c_m     = 1.0 * uF/cm2 # Capacitance
tau_ref = 2.5* ms # Refractory period
El      = -60 * mV # Leak potential
Vt      = -50 * mV # Spike Threshold
Vr      = -60 * mV # Spike Reset
Delta   = 2.5 * mV # Steepness of exponential approach to threshold 
tw      = 600. *ms # Adaptation



#Synapses
g_exc         =   6.  * nS# Excitatory conductance
g_inh         =   67. * nS# Inhibitory conductance
Ee            =  -70. * mV # Reversal potential (excitation)
Ei            =   0.* mV # Reversal potential (inhibition)
tau_exc       =   5. * ms # Synaptic time constant (excitatory)
tau_inh       =   10. * ms # Synaptic time constant (inhibitory)

Vcut = Vt + 5 * Delta #Taken from brian documentary


#ext_rate      = 100 * Hz                # Rate of the external source
ext_input     = 0 * nA #external current

# Pick an electrophysiological behaviour
a, b= 0.001 * usiemens, 0.04 * nA # Regular spiking (as in the paper)
# a, b= 0.001 * usiemens, 0.005 * nA # Bursting
# a, b= 0.001  *usiemens, 0 * nA # Fast spiking


### Equation for a Conductance-based IAF ####
eqs = Equations('''
dv/dt  = (gl*(El-v) + ge*(Ee-v)/ S +gi*(Ei-v)/ S + I_ext / S + gl*Delta*exp((v-Vt)/Delta) - wa/S)/c_m: volt
dwa/dt  = (a*(v - El)-wa)/tw : amp
dge/dt = -ge*(1./tau_exc) : siemens
dgi/dt = -gi*(1./tau_inh) : siemens
I_ext : amp                                             # external current
''')

P = NeuronGroup(100, eqs, threshold='v>Vt',  reset="v=Vr; wa+=b", method='euler')
P.I_ext = ext_input
P.v = -60*mV

Pe = P[:32]
Pi = P[32:]
Ce = Synapses(Pe, P,on_pre='ge+=g_exc') # 11mV
Ce.connect(p=0.02)
Ci = Synapses(Pi, P, on_pre='gi-=g_inh') # 8.5mV
Ci.connect(p=0.08)

trace = StateMonitor(P, 'v', record=0)
spikes = SpikeMonitor(P)
stim = StateMonitor(P, 'I_ext', record=0)

run(100 * ms)
P.I_ext = 0.25*nA
run(50 * ms)
P.I_ext = 0*nA
run(850 * ms)
# P.I_ext = -0.25*nA
# run(400 * ms)
# P.I_ext = 0*nA
# run(400 * ms)

In [ ]:
%matplotlib inline
import seaborn
seaborn.set()


# We draw nicer spikes
vm = trace[0].v[:]
for t in spikes.t:
    i = int(t / defaultclock.dt)
    vm[i] = 20*mV

plot(trace.t / ms, vm / mV)
xlabel('time (ms)')
ylabel('membrane potential (mV)')
xlim(0,1000)
title("Regular spiking neuron")

figure()

plot(trace.t/ms, stim[0].I_ext[:]/ nA)
title("Step-wise stimulus")
xlabel('time (ms)')
ylabel('Input current (nA)')


figure()
vlines(spikes.t/ms, spikes.i-0.5, spikes.i+0.5)
xlabel('time (ms)')
ylabel('NEURON ID')
title("Raster plot")
xlim(0, 1000)
ylim(-0.5, 40.5)
show()

In [ ]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')
    


In [ ]:
visualise_connectivity(Ce)

In [ ]:
visualise_connectivity(Ci)

# 2. Bursting mode

In [ ]:
from brian2 import *


### Cell parameters ###
S       = 20000.* cm2 # Area of membrane
gl      = 0.05 * msiemens/ cm2# Membrane time constant
c_m     = 1.0 * uF/cm2 # Capacitance
tau_ref = 2.5* ms # Refractory period
El      = -60 * mV # Leak potential
Vt      = -50 * mV # Spike Threshold
Vr      = -60 * mV # Spike Reset
Delta   = 2.5 * mV # Steepness of exponential approach to threshold 
tw      = 600. *ms # Adaptation



#Synapses
g_exc         =   6.  * nS# Excitatory conductance
g_inh         =   67. * nS# Inhibitory conductance
Ee            =  -70. * mV # Reversal potential (excitation)
Ei            =   0.* mV # Reversal potential (inhibition)
tau_exc       =   5. * ms # Synaptic time constant (excitatory)
tau_inh       =   10. * ms # Synaptic time constant (inhibitory)


#ext_rate      = 100 * Hz                # Rate of the external source
ext_input     = 0 * nA #external current

# Pick an electrophysiological behaviour
# a, b= 0.001 * usiemens, 0.04 * nA # Regular spiking (as in the paper)
a, b= 0.001 * usiemens, 0.005 * nA # Bursting
# a, b= 0.001  *usiemens, 0 * nA # Fast spiking


### Equation for a Conductance-based IAF ####
eqs = Equations('''
dv/dt  = (gl*(El-v) + ge*(Ee-v)/ S +gi*(Ei-v)/ S + I_ext / S + gl*Delta*exp((El-v)/Delta) - wa/S)/c_m: volt
dwa/dt  = (a*(El-v)-wa)/tw : amp
dge/dt = -ge*(1./tau_exc) : siemens
dgi/dt = -gi*(1./tau_inh) : siemens
I_ext : amp                                             # external current
''')

P = NeuronGroup(1, eqs, threshold='v>Vt',  reset="v=Vr; wa+=b", method='euler')
P.I_ext = ext_input
P.v = -60*mV

# Pe = P[:320]
# Pi = P[320:]
# Ce = Synapses(Pe, P, on_pre='ge+=g_exc') # 11mV
# Ce.connect(p=0.02)
# Ci = Synapses(Pi, P, on_pre='gi-=g_inh') # 8.5mV
# Ci.connect(p=0.08)

trace = StateMonitor(P, 'v', record=0)
spikes = SpikeMonitor(P)



run(20 * ms)
P.I_ext = 0.25*nA*5E7
run(100 * ms)
P.I_ext = 0*nA
run(200 * ms)


In [ ]:
%matplotlib inline
import seaborn
seaborn.set()


# We draw nicer spikes
vm = trace[0].v[:]
for t in spikes.t:
    i = int(t / defaultclock.dt)
    vm[i] = 20*mV

plot(trace.t / ms, vm / mV)
xlabel('time (ms)')
ylabel('membrane potential (mV)')
show()

figure()
vlines(spikes.t/ms, spikes.i-0.5, spikes.i+0.5)
show()

# 3. Fast Spiking

In [ ]:
from brian2 import *


### Cell parameters ###
S       = 20000.* cm2 # Area of membrane
gl      = 0.05 * msiemens/ cm2# Membrane time constant
c_m     = 1.0 * uF/cm**2 # Capacitance
tau_ref = 2.5* ms # Refractory period
El      = -60 * mV # Leak potential
Vt      = -50 * mV # Spike Threshold
Vr      = -60 * mV # Spike Reset
Delta   = 2.5 * mV # Steepness of exponential approach to threshold 
tw      = 600. *ms # Adaptation



#Synapses
g_exc         =   6.  * nS# Excitatory conductance
g_inh         =   67. * nS# Inhibitory conductance
Ee            =  -70. * mV # Reversal potential (excitation)
Ei            =   0.* mV # Reversal potential (inhibition)
tau_exc       =   5. * ms # Synaptic time constant (excitatory)
tau_inh       =   10. * ms # Synaptic time constant (inhibitory)


#ext_rate      = 100 * Hz                # Rate of the external source
ext_input     = 0 * nA #external current

# Pick an electrophysiological behaviour
# a, b= 0.001 * usiemens, 0.04 * nA # Regular spiking (as in the paper)
# a, b= 0.001 * usiemens, 0.005 * nA # Bursting
a, b= 0.001  *usiemens, 0 * nA # Fast spiking


### Equation for a Conductance-based IAF ####
eqs = Equations('''
dv/dt  = (gl*(El-v) + ge*(Ee-v)/ S +gi*(Ei-v)/ S + I_ext / S + gl*Delta*exp((El-v)/Delta) - wa/S)/c_m: volt
dwa/dt  = (a*(v - El)-wa)/tw : amp
dge/dt = -ge*(1./tau_exc) : siemens
dgi/dt = -gi*(1./tau_inh) : siemens
I_ext : amp                                             # external current
''')

P = NeuronGroup(1, eqs, threshold='v>Vt',  reset="v=Vr; wa+=b", method='euler')
P.I_ext = ext_input
P.v = -60*mV

# Pe = P[:320]
# Pi = P[320:]
# Ce = Synapses(Pe, P, on_pre='ge+=g_exc') # 11mV
# Ce.connect(p=0.02)
# Ci = Synapses(Pi, P, on_pre='gi-=g_inh') # 8.5mV
# Ci.connect(p=0.08)

trace = StateMonitor(P, 'v', record=0)
spikes = SpikeMonitor(P)



run(100 * ms)
P.I_ext = 0.25*nA
run(100 * ms)
P.I_ext = 0*nA
run(800 * ms)


In [ ]:
%matplotlib inline
import seaborn
seaborn.set()


# We draw nicer spikes
vm = trace[0].v[:]
for t in spikes.t:
    i = int(t / defaultclock.dt)
    vm[i] = 20*mV

plot(trace.t / ms, vm / mV)
xlabel('time (ms)')
ylabel('membrane potential (mV)')
show()

figure()
vlines(spikes.t/ms, spikes.i-0.5, spikes.i+0.5)
show()